In [1]:
import sys
sys.path.append('../')

from src.cmesrc.config import CMESRCV3_DB
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect(CMESRCV3_DB)
cur = conn.cursor()

data = pd.read_sql_query("""
SELECT C.cme_date, F.flare_id, F.flare_date, F.flare_class, F.flare_id, D.dimming_id, D.dimming_start_date FROM FINAL_CME_HARP_ASSOCIATIONS FCHA
INNER JOIN CMES_HARPS_EVENTS CHE
ON CHE.cme_id = FCHA.cme_id AND CHE.harpnum = FCHA.harpnum
INNER JOIN CMES C
ON FCHA.cme_id = C.cme_id
LEFT JOIN FLARES F
ON F.flare_id = CHE.flare_id
LEFT JOIN DIMMINGS D
ON D.dimming_id = CHE.dimming_id
INNER JOIN NOAA_HARPNUM_MAPPING NHM
ON NHM.harpnum = FCHA.harpnum
WHERE NHM.noaa = 12673
""", conn)

In [3]:
data

,cme_date,flare_id,flare_date,flare_class,flare_id,dimming_id,dimming_start_date
0,2017-09-04 19:00:05,13869,2017-09-04 18:12:00,M1.0,13869,9033.0,2017-09-04 18:17:59
1,2017-09-04 20:36:05,13872,2017-09-04 20:24:00,M5.5,13872,NaN,None
2,2017-09-06 09:48:06,13888,2017-09-06 09:00:00,X2.2,13888,NaN,None
3,2017-09-06 12:24:05,13889,2017-09-06 12:00:00,X9.3,13889,9037.0,2017-09-06 12:07:59
4,2017-09-07 10:24:06,13897,2017-09-07 10:12:00,M7.3,13897,NaN,None
5,2017-09-07 15:12:08,13899,2017-09-07 14:36:00,X1.3,13899,NaN,None
6,2017-09-07 18:48:05,13900,2017-09-07 18:24:00,C5.2,13900,NaN,None
7,2017-09-07 23:36:05,13903,2017-09-07 21:36:00,C5.4,13903,NaN,None
8,2017-09-08 03:24:06,13908,2017-09-08 02:24:00,M1.3,13908,NaN,None
9,2017-09-08 07:36:05,13911,2017-09-08 05:48:00,C8.3,13911,NaN,None


In [4]:
# Rename columns
# cme_date -> CME Date
# flare_id -> F. ID
# flare_date -> F. Date
# flare_class -> F. Class
# dimming_id -> D. ID
# dimming_start_date -> D. Date

data.columns = ['CME Date', 'F. ID', 'F. Date', 'F. Class', 'F. ID', 'D. ID', 'D. Date']

data

,CME Date,F. ID,F. Date,F. Class,F. ID,D. ID,D. Date
0,2017-09-04 19:00:05,13869,2017-09-04 18:12:00,M1.0,13869,9033.0,2017-09-04 18:17:59
1,2017-09-04 20:36:05,13872,2017-09-04 20:24:00,M5.5,13872,NaN,None
2,2017-09-06 09:48:06,13888,2017-09-06 09:00:00,X2.2,13888,NaN,None
3,2017-09-06 12:24:05,13889,2017-09-06 12:00:00,X9.3,13889,9037.0,2017-09-06 12:07:59
4,2017-09-07 10:24:06,13897,2017-09-07 10:12:00,M7.3,13897,NaN,None
5,2017-09-07 15:12:08,13899,2017-09-07 14:36:00,X1.3,13899,NaN,None
6,2017-09-07 18:48:05,13900,2017-09-07 18:24:00,C5.2,13900,NaN,None
7,2017-09-07 23:36:05,13903,2017-09-07 21:36:00,C5.4,13903,NaN,None
8,2017-09-08 03:24:06,13908,2017-09-08 02:24:00,M1.3,13908,NaN,None
9,2017-09-08 07:36:05,13911,2017-09-08 05:48:00,C8.3,13911,NaN,None


In [6]:
# Format Flare Date and Dimming Date to only include hour and minutes
data['F. Date'] = pd.to_datetime(data['F. Date'])
data['F. Date'] = data['F. Date'].dt.strftime('%H:%M')

data['D. Date'] = pd.to_datetime(data['D. Date'])
data['D. Date'] = data['D. Date'].dt.strftime('%H:%M')

# And for the CME Date take seconds out

data['CME Date'] = pd.to_datetime(data['CME Date'])
data['CME Date'] = data['CME Date'].dt.strftime('%Y-%m-%d %H:%M')

data

,CME Date,F. ID,F. Date,F. Class,F. ID,D. ID,D. Date
0,2017-09-04 19:00,13869,18:12,M1.0,13869,9033.0,18:17
1,2017-09-04 20:36,13872,20:24,M5.5,13872,NaN,NaN
2,2017-09-06 09:48,13888,09:00,X2.2,13888,NaN,NaN
3,2017-09-06 12:24,13889,12:00,X9.3,13889,9037.0,12:07
4,2017-09-07 10:24,13897,10:12,M7.3,13897,NaN,NaN
5,2017-09-07 15:12,13899,14:36,X1.3,13899,NaN,NaN
6,2017-09-07 18:48,13900,18:24,C5.2,13900,NaN,NaN
7,2017-09-07 23:36,13903,21:36,C5.4,13903,NaN,NaN
8,2017-09-08 03:24,13908,02:24,M1.3,13908,NaN,NaN
9,2017-09-08 07:36,13911,05:48,C8.3,13911,NaN,NaN


In [7]:
# Now into latex pretty table

print(data.to_latex(index=False, bold_rows=True))

\begin{tabular}{lrllrrl}
\toprule
        CME Date &  F. ID & F. Date & F. Class &  F. ID &  D. ID & D. Date \\
\midrule
2017-09-04 19:00 &  13869 &   18:12 &     M1.0 &  13869 & 9033.0 &   18:17 \\
2017-09-04 20:36 &  13872 &   20:24 &     M5.5 &  13872 &    NaN &     NaN \\
2017-09-06 09:48 &  13888 &   09:00 &     X2.2 &  13888 &    NaN &     NaN \\
2017-09-06 12:24 &  13889 &   12:00 &     X9.3 &  13889 & 9037.0 &   12:07 \\
2017-09-07 10:24 &  13897 &   10:12 &     M7.3 &  13897 &    NaN &     NaN \\
2017-09-07 15:12 &  13899 &   14:36 &     X1.3 &  13899 &    NaN &     NaN \\
2017-09-07 18:48 &  13900 &   18:24 &     C5.2 &  13900 &    NaN &     NaN \\
2017-09-07 23:36 &  13903 &   21:36 &     C5.4 &  13903 &    NaN &     NaN \\
2017-09-08 03:24 &  13908 &   02:24 &     M1.3 &  13908 &    NaN &     NaN \\
2017-09-08 07:36 &  13911 &   05:48 &     C8.3 &  13911 &    NaN &     NaN \\
2017-09-09 06:12 &  13930 &   04:24 &     M1.1 &  13930 &    NaN &     NaN \\
2017-09-09 16:24 &  1

/tmp/ipykernel_12593/3629072520.py:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(data.to_latex(index=False, bold_rows=True))
